# STEP BY STEP


1. Download DSM and DTM file
2. Open DSM and DTM file
3. Use API address to get coordinate of a specific location using http://loc.geopunt.be 
4. Check the bounds of the fileUse rasterio.bounds function to check the north, east, west, south bounds of the location
6. Use 'if function' if the building is located in the .tif file
7. Create 3D file

# Download DSM and DTM file

## DSM file

In [ ]:
#pip3 install beautifulsoup4
#pip3 install lxml


from bs4 import BeautifulSoup
import requests

url= 'https://www.geopunt.be/download?container=dhm-vlaanderen-ii-dsm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DSM,%20raster,%201m'
filetype = '.zip'

def web(url):
    return BeautifulSoup(requests.get(url).text, 'lxml')

for url_containing_link in web(url).find_all('a'):
    link = url_containing_link.get('href')
    if filetype in link:
        print(link)
        with open(url_containing_link.text, 'wb') as download_dsm:
            response = requests.get(link)
            download_dsm.write(response.content)
       

## DTM file

In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.geopunt.be/download?container=dhm-vlaanderen-ii-dtm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DTM,%20raster,%201m'
filetype = '.zip'

def web(url):
    return BeautifulSoup(requests.get(url).text, 'lxml')

for url_containing_link in web(url).find_all('a'):
    link = url_containing_link.get('href')
    if filetype in link:
        print(url_containing_link)
        with open(url_containing_link.text, 'wb') as download_dtm:
            response = requests.get(link)
            download_dtm.write(response.content)
            

## Check the .tif file

In [ ]:
import os
import rasterio
from rasterio.plot import show

#to open .tif wheter located in directory, subdirectory, or files.
#applied for dsm and dtm files

rootdir = '/Users/nadanadia/Desktop/Me/'
for subdir, dir, files in os.walk(rootdir):
    for f in files:
        if f.endswith('.tif'):
            get_file= os.path.join(subdir, f)
            print(get_file)

# Get the location and the bounds

1. Get the location with API address
2. Get the bounds by calling .bounds


In [1]:
import os
import rasterio
from rasterio.plot import show
import requests
import json


rootdir = '/Users/nadanadia/Desktop/Me/'
address = input("Please input the address: ")  
  
#http://loc.geopunt.be
response = requests.get(f"http://loc.geopunt.be/v4/location?q={address}")  
coordinate = json.loads(response.content)

if not coordinate['LocationResult']:
    raise ValueError('Please input the correct address')

x_axis = coordinate['LocationResult'][0]['Location']['X_Lambert72']
y_axis = coordinate['LocationResult'][0]['Location']['Y_Lambert72']

print(f'Address: {address}')
print(f'Coordinate of the location is: {float(x_axis), float(y_axis)}')

#make an empty list for file_descriptors to save bounds. description
file_descriptors = []

#to open .tif wheter located in directory, subdirectory, or files.
#applied for dsm and dtm files
for subdir, dir, files in os.walk(rootdir):
    for f in files:
        if f.endswith('.tif'):                
            path = os.path.join(subdir, f)
            file_descriptor = rasterio.open(path)
            
            #imagine if the image is inside a square. call .bounds[number] to know the square.
            #to get left side of the square (x_min), right side of the square (x_max), upper side of the square (y_max), and bottom side of the square (y_min)
            #check if the address is inside .tif file
           
            x_min = file_descriptor.bounds[0]
            x_max = file_descriptor.bounds[2]
            y_min = file_descriptor.bounds[1]
            y_max = file_descriptor.bounds[3]

            if x_min < x_axis < x_max and y_min < y_axis < y_max:
                #add bounds information to file_descriptors list
                file_descriptors.append(file_descriptor)
                print(path)
            else:
                #close file that is not suitable
                file_descriptor.close()
                    

Address: Zoerselhofdreef 40, 2980
Coordinate of the location is: (173172.83, 216574.07)
/Users/nadanadia/Desktop/Me/DHMVIIDTMRAS1m_k16/GeoTIFF/DHMVIIDTMRAS1m_k16.tif
/Users/nadanadia/Desktop/Me/DHMVIIDSMRAS1m_k16/GeoTIFF/DHMVIIDSMRAS1m_k16.tif


In [2]:
print(file_descriptors)

[<open DatasetReader name='/Users/nadanadia/Desktop/Me/DHMVIIDTMRAS1m_k16/GeoTIFF/DHMVIIDTMRAS1m_k16.tif' mode='r'>, <open DatasetReader name='/Users/nadanadia/Desktop/Me/DHMVIIDSMRAS1m_k16/GeoTIFF/DHMVIIDSMRAS1m_k16.tif' mode='r'>]


# Make Windows DSM and DTM

In [4]:
import random
import rasterio 
from rasterio.windows import Window 
from rasterio.windows import from_bounds

dsm_file = file_descriptors[1]
dtm_file = file_descriptors[0]

length = 40
xoff = x_axis
yoff = y_axis

left_new = xoff - length
right_new = xoff + length
bottom_new = yoff - length
top_new = yoff + length

window = from_bounds(left_new, bottom_new, right_new, top_new, transform=dsm_file.transform)
#transform dsm and dtm are same
print(window)



Window(col_off=11132.829999999987, row_off=1385.929999999993, width=80.0, height=80.0)


# Open DSM and DTM in numpy array

In [5]:
dsm_array = dsm_file.read( window=window)
dtm_array = dtm_file.read( window=window)

# Create 3D

In [6]:
chm_array = dsm_array - dtm_array
chm_array

array([[[ 0.        ,  0.        ,  0.        , ...,  9.09      ,
         10.030001  ,  6.029999  ],
        [ 0.        ,  0.        ,  0.        , ...,  6.1500015 ,
         10.129999  ,  8.969999  ],
        [ 0.        ,  0.        ,  0.        , ...,  5.5       ,
          6.0300007 ,  0.62000084],
        ...,
        [21.560001  , 18.43      , 22.099998  , ...,  0.        ,
          0.        ,  0.        ],
        [24.        , 22.43      , 16.81      , ...,  0.        ,
          0.        ,  0.        ],
        [20.43      , 19.46      , 19.98      , ...,  0.        ,
          0.        ,  0.        ]]], dtype=float32)

In [7]:
import plotly.io as pio
#pio.renderers.default = "browser"


import plotly.graph_objects as go

fig = go.Figure(data=go.Surface(z=chm_array[0]))

fig.show()

In [ ]:

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd



s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
url = (f"https://www.immoweb.be/en/classified/castle/for-sale/zoersel/2980/9397104?") 
driver.get(url)
soup = BeautifulSoup(driver.page_source, "lxml")

#price
for div_containing_price in soup.find_all("p", {"class":"classified__price"}):
    a = div_containing_price.find("span", {"class":"sr-only"})
    price = a.text
    print(price)